- dataset: **PASCAL VOC 12**
- Metrics: **Pixel Accuracy (PA)**, **Mean Pixel Accuracy (MPA)**, **Intersection over Union (IoU)**, **Mean Intersection over Union (MIoU)**
- Models: **context-based methods** DeepLabV3, HRNet
- Optimaizers: **Adam**, **AdaGrad**, **RmsProp**

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import VOCSegmentation
from torchvision.models.segmentation import deeplabv3_resnet101
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import torch.nn.functional as F

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Configuration
CONFIG = {
    'data_root': './data',
    'batch_size': 2,
    'num_epochs': 10,
    'learning_rate': 0.001,
    'num_classes': 21,  # PASCAL VOC has 20 classes + background
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'save_dir': './checkpoints',
    'results_dir': './results'
}

# Create directories if they don't exist
os.makedirs(CONFIG['save_dir'], exist_ok=True)
os.makedirs(CONFIG['results_dir'], exist_ok=True)

## **Data preprocessing and exploration**

Define transformations


In [2]:
def get_transforms():
    train_transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    val_transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    target_transform = transforms.Compose([
        transforms.Resize((256, 256), interpolation=transforms.InterpolationMode.NEAREST),
        transforms.ToTensor(),
    ])

    return train_transform, val_transform, target_transform
train_transform, val_transform, target_transform=get_transforms()

Data loading

In [3]:
def load_data():
    train_transform, val_transform, target_transform = get_transforms()

    train_dataset = VOCSegmentation(
        root=CONFIG['data_root'],
        year='2012',
        image_set='train',
        transform=train_transform,
        target_transform=target_transform,
        download=True
    )

    val_dataset = VOCSegmentation(
        root=CONFIG['data_root'],
        year='2012',
        image_set='val',
        transform=val_transform,
        target_transform=target_transform,
        download=True
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=CONFIG['batch_size'],
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=CONFIG['batch_size'],
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )

    return train_loader, val_loader
train_loader, val_loader=load_data()

100%|██████████| 2.00G/2.00G [01:26<00:00, 23.1MB/s]
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Data exploration

In [4]:
def explore_data(train_loader):
    print("Data Exploration:")

    # Get a batch
    images, masks = next(iter(train_loader))

    # Print shapes
    print(f"Image batch shape: {images.shape}")
    print(f"Mask batch shape: {masks.shape}")

    # Visualize some samples
    plt.figure(figsize=(16, 8))
    for i in range(min(4, CONFIG['batch_size'])):
        # Display image
        plt.subplot(2, 4, i+1)
        img = images[i].permute(1, 2, 0).numpy()
        img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
        img = np.clip(img, 0, 1)
        plt.imshow(img)
        plt.title(f"Image {i+1}")
        plt.axis('off')

        # Display mask
        plt.subplot(2, 4, i+5)
        mask = masks[i].squeeze().numpy()
        plt.imshow(mask)
        plt.title(f"Mask {i+1}")
        plt.axis('off')

    plt.tight_layout()
    plt.savefig(os.path.join(CONFIG['results_dir'], 'data_samples.png'))
    plt.close()

    # Class distribution
    class_counts = {}
    for _, mask in tqdm(train_loader, desc="Analyzing class distribution"):
        for class_id in range(CONFIG['num_classes']):
            pixels = (mask == class_id).sum().item()
            if class_id in class_counts:
                class_counts[class_id] += pixels
            else:
                class_counts[class_id] = pixels

    # Plot class distribution
    plt.figure(figsize=(12, 6))
    plt.bar(class_counts.keys(), class_counts.values())
    plt.xlabel('Class ID')
    plt.ylabel('Pixel Count')
    plt.title('Class Distribution in PASCAL VOC 2012')
    plt.savefig(os.path.join(CONFIG['results_dir'], 'class_distribution.png'))
    plt.close()

    print("Data exploration completed. Visualization saved.")
    return class_counts
class_counts=explore_data(train_loader)

Data Exploration:
Image batch shape: torch.Size([2, 3, 256, 256])
Mask batch shape: torch.Size([2, 1, 256, 256])


Analyzing class distribution: 100%|██████████| 732/732 [00:14<00:00, 49.40it/s]


Data exploration completed. Visualization saved.


## **Implementation of Models**

Models & optimizers definition

In [5]:
def get_deeplabv3():
    model = deeplabv3_resnet101(pretrained=True)
    model.classifier[4] = nn.Conv2d(256, CONFIG['num_classes'], kernel_size=1)
    return model

class HRNet(nn.Module):
    def __init__(self, num_classes=21):
        super(HRNet, self).__init__()
        # We'll use a simplified HRNet implementation
        # Initialize with a pretrained backbone (ResNet in this case)
        backbone = torchvision.models.resnet50(pretrained=True)
        self.layer0 = nn.Sequential(
            backbone.conv1,
            backbone.bn1,
            backbone.relu,
            backbone.maxpool
        )
        self.layer1 = backbone.layer1
        self.layer2 = backbone.layer2
        self.layer3 = backbone.layer3
        self.layer4 = backbone.layer4

        # High-resolution branches
        self.high_res_branch = nn.Sequential(
            nn.Conv2d(2048, 512, kernel_size=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )

        # Final classifier
        self.classifier = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, num_classes, kernel_size=1)
        )

    def forward(self, x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.high_res_branch(x)
        x = F.interpolate(x, scale_factor=16, mode='bilinear', align_corners=True)
        x = self.classifier(x)

        return {'out': x}

def get_model(model_name):
    if model_name.lower() == 'deeplabv3':
        return get_deeplabv3()
    elif model_name.lower() == 'hrnet':
        return HRNet(CONFIG['num_classes'])
    else:
        raise ValueError(f"Model {model_name} not supported!")

def get_optimizer(model, optimizer_name, lr=0.001):
    if optimizer_name.lower() == 'adam':
        return optim.Adam(model.parameters(), lr=lr)
    elif optimizer_name.lower() == 'adagrad':
        return optim.Adagrad(model.parameters(), lr=lr)
    elif optimizer_name.lower() == 'rmsprop':
        return optim.RMSprop(model.parameters(), lr=lr)
    else:
        raise ValueError(f"Optimizer {optimizer_name} not supported!")

Calculate gradients statistics

In [6]:
def get_gradient_stats(model):
    grad_stats = {}
    for name, param in model.named_parameters():
        if param.grad is not None:
            grad_stats[name] = {
                'mean': param.grad.mean().item(),
                'std': param.grad.std().item(),
                'min': param.grad.min().item(),
                'max': param.grad.max().item()
            }
    return grad_stats

Training loop

In [7]:
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    gradient_stats = []

    with tqdm(train_loader, desc="Training") as pbar:
        for images, masks in pbar:
            images = images.to(device)
            masks = masks.long().squeeze(1).to(device)

            # Forward pass
            outputs = model(images)['out']

            # Resize masks to match output size
            if outputs.shape[2:] != masks.shape[1:]:
                masks = F.interpolate(masks.unsqueeze(1).float(), size=outputs.shape[2:],
                                      mode='nearest').squeeze(1).long()

            loss = criterion(outputs, masks)

            # Rest of the function remains the same
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Collect gradient statistics
            grad_stats = get_gradient_stats(model)
            gradient_stats.append(grad_stats)

            # Update stats
            running_loss += loss.item()
            pbar.set_postfix({'loss': running_loss / (pbar.n + 1)})

    return running_loss / len(train_loader), gradient_stats


Evaluation metrics

In [8]:
def calculate_metrics(pred, target, num_classes=21):
    # Convert tensors to numpy arrays
    pred = pred.cpu().numpy()
    target = target.cpu().numpy()

    # Initialize metrics
    pixel_accuracy = 0.0
    class_accuracy = np.zeros(num_classes)
    iou = np.zeros(num_classes)

    # Calculate metrics for each image
    for i in range(pred.shape[0]):
        # Pixel Accuracy
        pixel_accuracy += np.mean(pred[i] == target[i])

        # Class Accuracy and IoU
        for cls in range(num_classes):
            pred_cls = pred[i] == cls
            target_cls = target[i] == cls

            # Skip if class not present in ground truth
            if np.sum(target_cls) == 0:
                continue

            # Class accuracy
            class_accuracy[cls] += np.sum(pred_cls & target_cls) / np.sum(target_cls)

            # IoU
            intersection = np.sum(pred_cls & target_cls)
            union = np.sum(pred_cls | target_cls)

            if union > 0:
                iou[cls] += intersection / union

    # Normalize by batch size
    pixel_accuracy /= pred.shape[0]

    # Count classes present in the batch
    class_counts = np.zeros(num_classes)
    for i in range(pred.shape[0]):
        for cls in range(num_classes):
            if np.sum(target[i] == cls) > 0:
                class_counts[cls] += 1

    # Normalize by class counts
    for cls in range(num_classes):
        if class_counts[cls] > 0:
            class_accuracy[cls] /= class_counts[cls]
            iou[cls] /= class_counts[cls]

    # Mean metrics
    mean_pixel_accuracy = np.mean([class_accuracy[c] for c in range(num_classes) if class_counts[c] > 0])
    mean_iou = np.mean([iou[c] for c in range(num_classes) if class_counts[c] > 0])

    metrics = {
        'PA': pixel_accuracy,
        'MPA': mean_pixel_accuracy,
        'IoU': iou,
        'MIoU': mean_iou
    }

    return metrics

Validation loop

In [9]:
def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_metrics = []

    with torch.no_grad():
        with tqdm(val_loader, desc="Validation") as pbar:
            for images, masks in pbar:
                images = images.to(device)
                masks = masks.long().squeeze(1).to(device)

                # Forward pass
                outputs = model(images)['out']

                # Resize masks to match output size
                if outputs.shape[2:] != masks.shape[1:]:
                    masks = F.interpolate(masks.unsqueeze(1).float(), size=outputs.shape[2:],
                                          mode='nearest').squeeze(1).long()

                loss = criterion(outputs, masks)

                # Get predictions
                preds = torch.argmax(outputs, dim=1)

                # Calculate metrics
                metrics = calculate_metrics(preds, masks, CONFIG['num_classes'])
                all_metrics.append(metrics)

                # Update stats
                running_loss += loss.item()
                pbar.set_postfix({'loss': running_loss / (pbar.n + 1), 'MIoU': metrics['MIoU']})

    # Aggregate metrics
    agg_metrics = {
        'PA': np.mean([m['PA'] for m in all_metrics]),
        'MPA': np.mean([m['MPA'] for m in all_metrics]),
        'MIoU': np.mean([m['MIoU'] for m in all_metrics]),
        'IoU': np.mean([m['IoU'] for m in all_metrics], axis=0)
    }

    return running_loss / len(val_loader), agg_metrics

Visualize predictions

In [10]:
def visualize_predictions(model, val_loader, device, num_samples=4):
    model.eval()
    images, masks = next(iter(val_loader))
    images = images.to(device)
    masks = masks.squeeze(1).to(device)

    with torch.no_grad():
        outputs = model(images)['out']
        preds = torch.argmax(outputs, dim=1)

    # Create a colormap
    cmap = plt.cm.get_cmap('tab20', CONFIG['num_classes'])

    plt.figure(figsize=(16, 12))
    for i in range(min(num_samples, images.shape[0])):
        # Original image
        plt.subplot(3, num_samples, i+1)
        img = images[i].cpu().permute(1, 2, 0).numpy()
        img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
        img = np.clip(img, 0, 1)
        plt.imshow(img)
        plt.title(f"Image {i+1}")
        plt.axis('off')

        # Ground truth
        plt.subplot(3, num_samples, i+num_samples+1)
        plt.imshow(masks[i].cpu().numpy(), cmap=cmap, vmin=0, vmax=CONFIG['num_classes']-1)
        plt.title(f"Ground Truth {i+1}")
        plt.axis('off')

        # Prediction
        plt.subplot(3, num_samples, i+2*num_samples+1)
        plt.imshow(preds[i].cpu().numpy(), cmap=cmap, vmin=0, vmax=CONFIG['num_classes']-1)
        plt.title(f"Prediction {i+1}")
        plt.axis('off')

    plt.tight_layout()
    plt.savefig(os.path.join(CONFIG['results_dir'], 'predictions.png'))
    plt.close()

Gradient analysis visualization

In [11]:
def visualize_gradients(gradient_stats, optimizer_name):
    # Extract data
    all_layers = list(gradient_stats[0].keys())
    selected_layers = all_layers[:3] + all_layers[-3:]  # First 3 and last 3 layers

    # Prepare data for plotting
    epochs = range(1, len(gradient_stats) + 1)

    # Plot gradient statistics over epochs
    plt.figure(figsize=(15, 10))

    for i, layer in enumerate(selected_layers):
        mean_values = [stats[layer]['mean'] for stats in gradient_stats]
        std_values = [stats[layer]['std'] for stats in gradient_stats]

        plt.subplot(2, 3, i+1)
        plt.plot(epochs, mean_values, 'b-', label='Mean')
        plt.fill_between(epochs,
                         [m - s for m, s in zip(mean_values, std_values)],
                         [m + s for m, s in zip(mean_values, std_values)],
                         alpha=0.3)
        plt.title(f'Layer: {layer.split(".")[-1]}')
        plt.xlabel('Epoch')
        plt.ylabel('Gradient Value')
        plt.legend()

    plt.suptitle(f'Gradient Analysis - {optimizer_name} Optimizer')
    plt.tight_layout()
    plt.savefig(os.path.join(CONFIG['results_dir'], f'gradient_analysis_{optimizer_name}.png'))
    plt.close()

## **Experimentation**

Training & evaluation pipeline

In [12]:
def run_experiment(model_name, optimizer_name):
    print(f"\n{'='*50}")
    print(f"Running experiment: Model={model_name}, Optimizer={optimizer_name}")
    print(f"{'='*50}\n")

    # Load data
    train_loader, val_loader = load_data()

    # Create model
    model = get_model(model_name)
    model = model.to(CONFIG['device'])

    # Create optimizer
    optimizer = get_optimizer(model, optimizer_name, CONFIG['learning_rate'])

    # Loss function
    criterion = nn.CrossEntropyLoss()

    # Initialize trackers
    best_miou = 0.0
    train_losses = []
    val_losses = []
    metrics_history = []
    all_gradient_stats = []

    # Training loop
    for epoch in range(CONFIG['num_epochs']):
        print(f"\nEpoch {epoch+1}/{CONFIG['num_epochs']}")

        # Train
        train_loss, grad_stats = train(model, train_loader, optimizer, criterion, CONFIG['device'])
        train_losses.append(train_loss)
        all_gradient_stats.append(grad_stats[-1])  # Save the last batch gradient stats

        # Validate
        val_loss, metrics = validate(model, val_loader, criterion, CONFIG['device'])
        val_losses.append(val_loss)
        metrics_history.append(metrics)

        # Print metrics
        print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
        print(f"PA: {metrics['PA']:.4f}, MPA: {metrics['MPA']:.4f}, MIoU: {metrics['MIoU']:.4f}")

        # Save best model
        if metrics['MIoU'] > best_miou:
            best_miou = metrics['MIoU']
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'metrics': metrics
            }, os.path.join(CONFIG['save_dir'], f"{model_name}_{optimizer_name}_best.pth"))
            print(f"New best model saved! MIoU: {best_miou:.4f}")

    # Visualize training progress
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Loss Curves')

    plt.subplot(1, 2, 2)
    plt.plot([m['PA'] for m in metrics_history], label='PA')
    plt.plot([m['MPA'] for m in metrics_history], label='MPA')
    plt.plot([m['MIoU'] for m in metrics_history], label='MIoU')
    plt.xlabel('Epoch')
    plt.ylabel('Metric Value')
    plt.legend()
    plt.title('Performance Metrics')

    plt.tight_layout()
    plt.savefig(os.path.join(CONFIG['results_dir'], f'{model_name}_{optimizer_name}_training.png'))
    plt.close()

    # Load best model for final evaluation
    checkpoint = torch.load(os.path.join(CONFIG['save_dir'], f"{model_name}_{optimizer_name}_best.pth"), weights_only=False)
    model.load_state_dict(checkpoint['model_state_dict'])

    # Final validation
    _, final_metrics = validate(model, val_loader, criterion, CONFIG['device'])

    # Visualize some predictions
    visualize_predictions(model, val_loader, CONFIG['device'])

    # Visualize gradient analysis
    visualize_gradients(all_gradient_stats, optimizer_name)

    return final_metrics

In [ ]:
models = ['DeepLabV3', 'HRNet']
optimizers = ['Adam', 'AdaGrad', 'RmsProp']

results = []

for model_name in models:
        for optimizer_name in optimizers:
            metrics = run_experiment(model_name, optimizer_name)


Running experiment: Model=DeepLabV3, Optimizer=Adam



/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet101_coco-586e9e4e.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet101_coco-586e9e4e.pth
100%|██████████| 233M/233M [00:01<00:00, 167MB/s]



Epoch 1/10


Validation: 100%|██████████| 725/725 [01:08<00:00, 10.58it/s, loss=0.219, MIoU=0.494]


Train Loss: 0.2787, Val Loss: 0.2187
PA: 0.9447, MPA: 0.5000, MIoU: 0.4724
New best model saved! MIoU: 0.4724

Epoch 2/10


...

Validation: 100%|██████████| 725/725 [01:09<00:00, 10.46it/s, loss=0.211, MIoU=0.494]


Train Loss: 0.2160, Val Loss: 0.2103
PA: 0.9447, MPA: 0.5000, MIoU: 0.4724

Epoch 7/10


Validation: 100%|██████████| 725/725 [01:09<00:00, 10.47it/s, loss=0.21, MIoU=0.494]


Train Loss: 0.2136, Val Loss: 0.2100
PA: 0.9447, MPA: 0.5000, MIoU: 0.4724

Epoch 8/10


Training:  15%|█▌        | 112/732 [00:35<03:10,  3.26it/s, loss=0.227]

## **Evaluation of model performance**

In [ ]:
for model_name in models:
        for optimizer_name in optimizers:
            results.append({
                'Model': model_name,
                'Optimizer': optimizer_name,
                'PA': metrics['PA'],
                'MPA': metrics['MPA'],
                'MIoU': metrics['MIoU']
            })
            import gc
            gc.collect()
            torch.cuda.empty_cache()

## **Comparison of results and discussion**

In [ ]:
def compare_experiments(results):
    # Create comparison dataframe
    df = pd.DataFrame(results)

    # Plot comparison
    plt.figure(figsize=(15, 10))

    # PA comparison
    plt.subplot(2, 2, 1)
    sns.barplot(x='Model', y='PA', hue='Optimizer', data=df)
    plt.title('Pixel Accuracy Comparison')

    # MPA comparison
    plt.subplot(2, 2, 2)
    sns.barplot(x='Model', y='MPA', hue='Optimizer', data=df)
    plt.title('Mean Pixel Accuracy Comparison')

    # MIoU comparison
    plt.subplot(2, 2, 3)
    sns.barplot(x='Model', y='MIoU', hue='Optimizer', data=df)
    plt.title('Mean IoU Comparison')

    # Best model-optimizer combination
    plt.subplot(2, 2, 4)
    best_idx = df['MIoU'].idxmax()
    best_model = df.loc[best_idx, 'Model']
    best_optim = df.loc[best_idx, 'Optimizer']

    plt.text(0.5, 0.5, f"Best Model: {best_model}\nBest Optimizer: {best_optim}\nMIoU: {df.loc[best_idx, 'MIoU']:.4f}",
             ha='center', va='center', fontsize=14)
    plt.axis('off')

    plt.tight_layout()
    plt.savefig(os.path.join(CONFIG['results_dir'], 'model_optimizer_comparison.png'))
    plt.close()

    # Save results to CSV
    df.to_csv(os.path.join(CONFIG['results_dir'], 'experiment_results.csv'), index=False)

    return df

In [ ]:
comparison = compare_experiments(results)
print("\nExperiment Results:")
print(comparison)

print(f"\nAll experiments completed! Results saved to {CONFIG['results_dir']}")